# Orchestration Notebook

This notebook illustrates the Orchestrator component, which manages the multi-agent workflow.

## Purpose
The Orchestrator coordinates the specialized agents to deliver a complete solution. This notebook demonstrates:

1.  **Workflow Management**: Chaining the Designer, Optimizer, Validator, and Educational agents.
2.  **End-to-End Execution**: Processing a user query through the full pipeline: Design -> Optimize -> Validate -> Explain.
3.  **Error Handling**: Managing failures and feedback between agents.

## Usage
Use this notebook to run the complete multi-agent system for high-quality code generation.


In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.cirq_rag_code_assistant.config import get_config, setup_logging
from src.rag.knowledge_base import KnowledgeBase
from src.rag.retriever import Retriever
from src.rag.generator import Generator
from src.agents.designer import DesignerAgent
from src.agents.optimizer import OptimizerAgent
from src.agents.validator import ValidatorAgent
from src.agents.educational import EducationalAgent
from src.orchestration.orchestrator import Orchestrator

# Setup logging
setup_logging()

### Initialize Components
We need to initialize all agents and the orchestrator.

In [ ]:
# Initialize RAG
kb = KnowledgeBase()
retriever = Retriever(knowledge_base=kb)
generator = Generator(retriever=retriever)

# Initialize Agents
designer = DesignerAgent(retriever=retriever, generator=generator)
optimizer = OptimizerAgent()
validator = ValidatorAgent()
educational = EducationalAgent(retriever=retriever)

# Initialize Orchestrator
orchestrator = Orchestrator(
    designer=designer,
    optimizer=optimizer,
    validator=validator,
    educational=educational
)
print("Orchestrator initialized with all agents.")

### Run Full Workflow
Let's run the complete pipeline for a request.

In [ ]:
query = "Create a quantum circuit for Grover's algorithm with 2 qubits"

try:
    result = orchestrator.generate_code(
        query=query,
        algorithm="grover",
        optimize=True,
        validate=True,
        explain=True
    )
    
    if result['success']:
        print("Workflow Completed Successfully!")
        
        print("\nFinal Code:")
        print("-" * 40)
        print(result.get('optimized_code') or result.get('code'))
        print("-" * 40)
        
        if result.get('validation'):
            print(f"\nValidation Passed: {result['validation']['validation_passed']}")
            
        if result.get('explanation'):
            print("\nExplanation Overview:")
            print(result['explanation']['overview'])
            
    else:
        print("Workflow Failed.")
        print("Errors:", result.get('errors'))
        
except Exception as e:
    print(f"Error executing workflow: {e}")